In [1]:
import sys
sys.version

'3.8.3 (default, Jul  2 2020, 11:26:31) \n[Clang 10.0.0 ]'

In [2]:
!pip install self_attention_cv

In [ ]:
 !cp /content/drive/MyDrive/SalTran/* .

In [3]:
import time
import os
import copy
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from torch.optim import lr_scheduler
from torch.utils.data import Dataset, Subset
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
from self_attention_cv.vit import ResNet50ViT
from utils import corr_coeff, kld_loss, nss, train_val_dataset
from saltran import SalTran


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

zsh:1: command not found: nvidia-smi


In [5]:
torch.cuda.empty_cache()
torch.autograd.set_detect_anomaly(True)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

In [7]:
import importlib
import mit1003

In [8]:
importlib.reload(mit1003)

<module 'mit1003' from '/Users/eax/worx/deeplearning/attn/saltran/mit1003.py'>

In [9]:
dataset = mit1003.MIT1003Data('./MIT1003/', 'ALLSTIMULI', 'ALLFIXATIONMAPS')
ds = train_val_dataset(dataset)
dataloaders = {x: torch.utils.data.DataLoader(ds[x], batch_size=1,
                        shuffle=True, num_workers=0)
        for x in ['train', 'val']}

dataset_sizes = {x: len(ds[x]) for x in ['train', 'val']}
print(dataset_sizes)

{'train': 501, 'val': 502}


In [10]:
train = ds['train']
i = train[32]['img']
f = train[32]['fix']
m = train[32]['map']
n = train[32]['name']
nf = train[32]['fname']
nm = train[32]['mname']
print(f'{i.min()}, {i.max()}, {i.mean()}, {i.std()}')
print(f'{m.min()}, {m.max()}, {m.mean()}, {m.std()}')
print(f'{n}')
print(f'{nf}')
print(f'{nm}')

-2.1179039478302, 2.640000104904175, 0.2665327787399292, 1.3819854259490967
0.0, 0.9607843160629272, 0.031963273882865906, 0.10231802612543106
./MIT1003/ALLSTIMULI/i1429004931.jpeg
./MIT1003/ALLFIXATIONMAPS/i1429004931_fixPts.jpg
./MIT1003/ALLFIXATIONMAPS/i1429004931_fixMap.jpg


/Users/eax/miniconda3/envs/b/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [ ]:
!pip install opencv-python

In [25]:
from torchvision import transforms
from PIL import Image
from utils import normalize_tensor
import cv2
i = Image.open(n)#.convert('L')
ij = transforms.Compose([
  transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.BILINEAR),
  transforms.ToTensor()
])
ixj = ij(i)
print(ixj.min(), ixj.max(), ixj.mean(), ixj.std())
it = transforms.Compose([
  transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.BILINEAR),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
ix = it(i)
print(ix.min(), ix.max(), ix.mean(), ix.std())
im = Image.open(nm)
mt = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])
mx = mt(im)
fi = Image.open(nf).convert('1')
ft = transforms.Compose([
  transforms.Resize((256, 256)),
  transforms.ToTensor()
])
ixf = ft(fi)
print(mx.min(), mx.max(), mx.mean(), mx.std())


tensor(0.) tensor(1.) tensor(0.5093) tensor(0.3133)
tensor(-2.1179) tensor(2.6400) tensor(0.2666) tensor(1.3760)
tensor(0.) tensor(0.9608) tensor(0.0320) tensor(0.1023)


In [26]:
import saltran
importlib.reload(saltran)

<module 'saltran' from '/Users/eax/worx/deeplearning/attn/saltran/saltran.py'>

In [27]:
model = saltran.SalTran()
model.to(device)

SalTran(
  (xformer): ResNet50ViT(
    (model): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [28]:
from tqdm import tqdm
from tqdm import trange

In [ ]:
# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
best_f1 = 0.0

barzero = tqdm(range(5), position=0, leave=True)
print("---")

for epoch in barzero:
  for phase in ['train', 'val']:
    if phase == 'train':
      model.train()
    else:
      model.eval()

    running_loss = 0.0
    bar = tqdm(dataloaders[phase], desc=phase, position=0, leave=True)
    for data in bar:
      imgs = data['img'].float().to(device)
      fiks = data['fix'].to(device)
      maps = data['map'].float().to(device)
      name = data['name']
      optimizer.zero_grad()

      with torch.set_grad_enabled(phase == 'train'):
        outs = model(imgs)
        losses = []
        kl = kld_loss(outs, maps)
        ns = nss(outs, fiks)
        cc = corr_coeff(outs, maps)
        print(f'kl is nan for {name}') if math.isnan(kl) else None
        print(f'nss is nan for {name}') if math.isnan(ns) else None
        print(f'cc is nan for {name}') if math.isnan(cc) else None
        losses.append(kl)
        losses.append(ns)
        losses.append(cc)
        losses = [l.mean(1).mean(0) for l in losses]
        loss_weights = [1, 1, 1]
        loss = sum(weight * l for weight, l in
                              zip(loss_weights, losses))
        if phase == 'train':
          bar.set_description(f'{outs.min():.2f}, {outs.max():.2f}, {outs.mean():.2f}, {outs.std():.2f} - loss: {loss.item():.5f}')
          loss.backward()
          optimizer.step()

      running_loss += loss.item() * imgs.size(0)
    

    if phase == 'train':
      scheduler.step()

  epoch_loss = running_loss / dataset_sizes[phase]
  print(f'\nEpoch Loss: {epoch_loss:.5f} Running Loss: {running_loss:.5f}')

time_elapsed = time.time() - since
print(time_elapsed)

In [ ]:
!cp ./*.py /content/drive/MyDrive/SalTran/

In [ ]:
def visualize_model(model, num_images=10):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, inputs in enumerate(dataloaders['val']):
            inputs = inputs['img'].to(device)

            outputs = model(inputs)
            # _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted')
                imshow(outputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp[:, :, 0], cmap='gray')
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
visualize_model(model)